In [3]:
from ortools.linear_solver import pywraplp

def solver(N_classes, N_rooms, t, g, s, c):
    # Tạo một solver MIP
    solver = pywraplp.Solver.CreateSolver('SCIP')

    # Khai báo các biến
    X = [solver.IntVar(1, 60, f'X_{i}') for i in range(N_classes)]
    Y = [[solver.BoolVar(f'Y_{i}_{j}') for j in range(N_rooms)] for i in range(N_classes)]

    # Ràng buộc: Trong một phòng học trong một tiết chỉ có một lớp học.
    for i1 in range(N_classes):
        for i2 in range(i1 + 1, N_classes):
            for j in range(N_rooms):
                overlap = solver.BoolVar(f'overlap_{i1}_{i2}_{j}')
                solver.Add(overlap >= Y[i1][j] + Y[i2][j] - 1)
                solver.Add(overlap <= Y[i1][j])
                solver.Add(overlap <= Y[i2][j])
                choose_class = solver.BoolVar(f'choose_class_{i1}_first_{i2}')
                
                # Ràng buộc với điều kiện chọn 1 trong 2
                solver.Add(X[i1] + t[i1] <= X[i2] + 60 * (1 - overlap) + 60 * (1 - choose_class))
                solver.Add(X[i2] + t[i2] <= X[i1] + 60 * (1 - overlap) + 60 * choose_class)
                

    # Ràng buộc: Trong một tiết, một giáo viên chỉ dạy một lớp.
    for i1 in range(N_classes):
        for i2 in range(i1 + 1, N_classes):
            if g[i1] == g[i2]:
                choose_teacher = solver.BoolVar(f'choose_teacher_{i1}_{i2}')
                solver.Add(X[i1] + t[i1] <= X[i2]  + 60 * (1 - choose_teacher))
                solver.Add(X[i2] + t[i2] <= X[i1]  + 60 * choose_teacher)


    valid_ranges = [(1, 6), (7, 12), (13, 18), (19, 24), (25, 30), (31, 36), (37, 42), (43, 48), (49, 54), (55, 60)]
    for i in range(N_classes):
        bool_var_ranges = []
        for (start, end) in valid_ranges:
            bool_var = solver.BoolVar(f'b_{i}_{start}_{end}')
            bool_var_ranges.append(bool_var)
            solver.Add(X[i] >= start - 60 * (1 - bool_var))
            solver.Add(X[i] + t[i] - 1 <= end + 60 * (1 - bool_var))
        solver.Add(sum(bool_var_ranges) >= 1)
    
    # Ràng buộc: Thời điểm kết thúc của lớp học không được vượt quá tiết 60
    for i in range(N_classes):
        solver.Add(X[i] + t[i] - 1 <= 60)

    # Ràng buộc: 1 lớp học chỉ có thể diễn ra trong tối đa 1 phòng học
    for i in range(N_classes):
        solver.Add(sum(Y[i][j] for j in range(N_rooms)) <= 1)

    # Ràng buộc: Giới hạn phòng học
    for i in range(N_classes):
        for j in range(N_rooms):
            solver.Add(s[j] >= c[i] * Y[i][j])

    # Hàm mục tiêu: Tối đa hóa số lớp học được xếp
    Z = solver.Sum(Y[i][j] for j in range(N_rooms) for i in range(N_classes))
    solver.Maximize(Z)

    status = solver.Solve()

    # Kiểm tra kết quả và in ra giá trị của các biến
    if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
        print(int(solver.Objective().Value()))
        for i in range(N_classes):
            for j in range(N_rooms):
                if Y[i][j].solution_value() == 1:
                    print(f'{i + 1} {int(X[i].solution_value())} {j + 1}')
    else:
        print('Không tìm thấy giải pháp khả thi.')
N_classes, N_rooms = map(int, input().split())
t = [0 for _ in range(N_classes)]
g = [0 for _ in range(N_classes)]
c = [0 for _ in range(N_classes)]
s = [0 for _ in range(N_rooms)]
for i in range(N_classes):
    t[i], g[i], c[i] = map(int, input().split())
s = list(map(int, input().split()))
solver(N_classes, N_rooms, t, g, s, c)

 10 2
 4 1 15
 4 1 18
 4 1 15
 2 2 18
 4 2 11
 3 1 15
 2 2 27
 3 2 18
 4 1 13
 3 1 10
 20 20


9
1 9 1
2 1 1
3 31 1
4 49 1
5 55 2
6 58 1
8 13 1
9 13 2
10 55 1
